---
jupyter: Python3
date: 2019-10-11
title: Bigquery_사분위수 구하기
toc: true
categories: Bigquery
author: limyj0708
comments:
  giscus: 
    repo: quarto-dev/quarto-docs
format:
    html:
        page-layout: full
---

- 사분위수를 Bigquery에서 미리 구해서 시각화하고 싶을 떄가 있다.
- 두 가지 방법이 있는데, 아래와 같다.

## APPROX_QUANTILES
- [근사치 집계 함수](https://cloud.google.com/bigquery/docs/reference/standard-sql/approximate_aggregate_functions?hl=ko)
  - 근사치 집계 함수는 메모리 사용량과 시간 면에서 확장 가능하지만 정확한 결과가 아닌 근사치 결과를 산출합니다. 이러한 함수는 일반적으로 COUNT(DISTINCT ...)와 같은 정확한 집계 함수보다 적은 메모리를 사용하지만 통계상의 불확실성 또한 존재합니다. 따라서 근사치 집계는 선형 메모리 사용량이 비효율적이거나 데이터가 이미 근사치인 대용량 데이터 스트림에 적합합니다.
- 예를 들어, skill_score라는 수치 컬럼의 사분위수를 구한다고 해 보자.
    
```MySQL
min(skill_enhance_score_sum) as score_min
,APPROX_QUANTILES(skill_enhance_score_sum, 100)[OFFSET(25)] AS q_1 -- 100조각 내서 25번째, Q1
,APPROX_QUANTILES(skill_enhance_score_sum, 100)[OFFSET(50)] AS median
,APPROX_QUANTILES(skill_enhance_score_sum, 100)[OFFSET(75)] AS q_3
,max(skill_enhance_score_sum) as score_max
```


## PERCENTILE_CONT
- [탐색 함수](https://cloud.google.com/bigquery/docs/reference/standard-sql/navigation_functions?hl=ko#last_value)
- value_expression의 지정된 백분위수 값을 선형 보간으로 계산합니다.
- 예를 들어, skill_score라는 수치 컬럼의 사분위수를 구한다고 해 보자.
    
```MySQL
PERCENTILE_CONT(skill_enhance_score_sum, 0) OVER() as score_min
,PERCENTILE_CONT(skill_enhance_score_sum, 0.25) OVER() AS q_1
,PERCENTILE_CONT(skill_enhance_score_sum, 0.5) OVER() AS median
,PERCENTILE_CONT(skill_enhance_score_sum, 0.75) OVER() AS q_3
,PERCENTILE_CONT(skill_enhance_score_sum, 1) OVER() as score_max
```

- OVER 절은 [윈도우 함수 사용법](https://cloud.google.com/bigquery/docs/reference/standard-sql/window-function-calls?hl=ko)을 참고하자.


## 데이터에 NULL이 있는 경우에는?
  - 각 함수 설명에 NULL이 있는 경우 어떻게 되는지 예시가 나와 있으므로, 참고하여 사용하자.
  - [근사치 집계 함수](https://cloud.google.com/bigquery/docs/reference/standard-sql/approximate_aggregate_functions?hl=ko)
  - [탐색 함수](https://cloud.google.com/bigquery/docs/reference/standard-sql/navigation_functions?hl=ko#last_value)